<a href="https://colab.research.google.com/github/Rahul19982022/anime-recommendation-system-exp/blob/main/archives/3_content_encodings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q sentence-transformers==4.1.0
!pip install -q einops==0.8.1

In [1]:
import pandas as pd
import numpy as np
import os
import pickle

In [3]:
from sentence_transformers import SentenceTransformer

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# put path of your project folder here
project_path = '/content/drive/MyDrive/anime_proj'
os.chdir(project_path)

In [ ]:
anime = pd.read_csv('datasets/kaggle_dataset/anime.csv')
anime_with_synopsis = pd.read_csv('datasets/kaggle_dataset/anime_with_synopsis.csv')
plot_df = pd.read_csv('datasets/created_datasets/plot_df.csv')

In [ ]:
anime.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17562 entries, 0 to 17561
Data columns (total 35 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   MAL_ID         17562 non-null  int64 
 1   Name           17562 non-null  object
 2   Score          17562 non-null  object
 3   Genres         17562 non-null  object
 4   English name   17562 non-null  object
 5   Japanese name  17562 non-null  object
 6   Type           17562 non-null  object
 7   Episodes       17562 non-null  object
 8   Aired          17562 non-null  object
 9   Premiered      17562 non-null  object
 10  Producers      17562 non-null  object
 11  Licensors      17562 non-null  object
 12  Studios        17562 non-null  object
 13  Source         17562 non-null  object
 14  Duration       17562 non-null  object
 15  Rating         17562 non-null  object
 16  Ranked         17562 non-null  object
 17  Popularity     17562 non-null  int64 
 18  Members        17562 non-n

In [ ]:
rem_genre = anime[anime['Genres'].str.contains('Hentai')]['MAL_ID'].tolist()
music_cat = anime[anime['Type'] == 'Music']['MAL_ID'].tolist()
unknown_cat = anime[anime['Type'] == 'Unknown']['MAL_ID'].tolist()

anime = anime[~anime['MAL_ID'].isin(rem_genre + music_cat + unknown_cat)]
anime.reset_index(drop = True, inplace = True)

In [ ]:
anime.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14708 entries, 0 to 14707
Data columns (total 35 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   MAL_ID         14708 non-null  int64 
 1   Name           14708 non-null  object
 2   Score          14708 non-null  object
 3   Genres         14708 non-null  object
 4   English name   14708 non-null  object
 5   Japanese name  14708 non-null  object
 6   Type           14708 non-null  object
 7   Episodes       14708 non-null  object
 8   Aired          14708 non-null  object
 9   Premiered      14708 non-null  object
 10  Producers      14708 non-null  object
 11  Licensors      14708 non-null  object
 12  Studios        14708 non-null  object
 13  Source         14708 non-null  object
 14  Duration       14708 non-null  object
 15  Rating         14708 non-null  object
 16  Ranked         14708 non-null  object
 17  Popularity     14708 non-null  int64 
 18  Members        14708 non-n

In [ ]:
anime_with_synopsis.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16214 entries, 0 to 16213
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   MAL_ID     16214 non-null  int64 
 1   Name       16214 non-null  object
 2   Score      16214 non-null  object
 3   Genres     16214 non-null  object
 4   sypnopsis  16206 non-null  object
dtypes: int64(1), object(4)
memory usage: 633.5+ KB


In [ ]:
anime = anime.copy()

In [ ]:
anime['synopsis'] = anime['MAL_ID'].map(anime_with_synopsis.set_index('MAL_ID')['sypnopsis'])
anime['synopsis']

,synopsis
0,"In the year 2071, humanity has colonized sever..."
1,"other day, another bounty—such is the life of ..."
2,"Vash the Stampede is the man with a $$60,000,0..."
3,ches are individuals with special powers like ...
4,It is the dark century and the people are suff...
...,...
14703,No synopsis information has been added to this...
14704,ko is a typical high school student whose life...
14705,Sequel to Higurashi no Naku Koro ni Gou .
14706,New Yama no Susume anime.


In [ ]:
anime['synopsis_edited'] = anime['synopsis'].str.replace('\n', ' ')

In [ ]:
anime['synopsis_edited'].value_counts().head(10)

,count
synopsis_edited,
No synopsis information has been added to this title. Help improve our database by adding a synopsis here .,678
No synopsis has been added for this series yet. Click here to update this information.,35
Film by Takashi Ito.,13
Furukawa Taku film.,13
short animation by Taku Furukawa.,10
short film by Okamoto Tadanari.,8
Short animation by Rapparu.,8
short puppet animation movie by Tadahito Mochinaga.,6
Short film by Kurosaka Keita.,6


In [ ]:
remove_synopsis = anime['synopsis_edited'].value_counts().head(2).index.tolist()
remove_synopsis

['No synopsis information has been added to this title. Help improve our database by adding a synopsis here .',
 'No synopsis has been added for this series yet. Click here to update this information.']

In [ ]:
anime.loc[anime['synopsis_edited'].isin(remove_synopsis), 'synopsis_edited'] = np.nan

In [ ]:
anime['synopsis_edited'].value_counts().head(5)

,count
synopsis_edited,
Film by Takashi Ito.,13
Furukawa Taku film.,13
short animation by Taku Furukawa.,10
short film by Okamoto Tadanari.,8
Short animation by Rapparu.,8


In [ ]:
anime['synopsis_edited'].isna().sum()

np.int64(721)

In [ ]:
anime['synopsis_edited'][anime['synopsis_edited'].notna()].str.split(' ').apply(lambda x: len(x)).value_counts().sort_index().head(20)

,count
synopsis_edited,
2,32
3,48
4,148
5,264
6,269
7,280
8,300
9,272
10,237


In [ ]:
synp_not_na = anime['synopsis_edited'][anime['synopsis_edited'].notna()]
synp_not_na

,synopsis_edited
0,"In the year 2071, humanity has colonized sever..."
1,"other day, another bounty—such is the life of ..."
2,"Vash the Stampede is the man with a $$60,000,0..."
3,ches are individuals with special powers like ...
4,It is the dark century and the people are suff...
...,...
14701,The first astronaut in human history was a vam...
14704,ko is a typical high school student whose life...
14705,Sequel to Higurashi no Naku Koro ni Gou .
14706,New Yama no Susume anime.


In [ ]:
synp_not_na[synp_not_na.str.split(' ').apply(lambda x: 31 <= len(x) <= 40)].iloc[516]

"Based on Miura Ayako's literary work. Tells the story of Nagano Nobuo from his childhood in Tokyo, through a gradual journey of self-discovery and acceptance of the Christian faith, to his death in Shiokari Pass in Hokkaido. (Source: Jclglobal)"

In [ ]:
short_sysp = synp_not_na[synp_not_na.str.split(' ').apply(lambda x: len(x) <= 30)].index.tolist()
len(short_sysp)

4839

In [ ]:
anime.loc[short_sysp, 'synopsis_edited'] = np.nan

In [ ]:
anime['synopsis_edited'].isna().sum()

np.int64(5560)

In [ ]:
anime['plot'] = anime['MAL_ID'].map(plot_df.set_index('MAL_ID')['Plot'])
anime['plot']

,plot
0,"In the year 2071, roughly fifty years after an..."
1,"A few days before Halloween on Mars, Faye witn..."
2,"In the 32nd century, a man known as ""Vash the ..."
3,"Robin Sena is a ""craft user"", born in Japan an..."
4,"Set in the Dark Ages, a term referring to the ..."
...,...
14703,NaN
14704,High school student Miko Yotsuya has the unfor...
14705,NaN
14706,NaN


In [ ]:
anime['plot_edited'] = anime['plot']

In [ ]:
anime['plot_edited'][anime['plot_edited'].notna()].str.split(' ').apply(lambda x: len(x)).value_counts().sort_index().head(20)

,count
plot_edited,
5,1
8,1
9,1
10,1
13,1
14,2
15,2
16,3
18,3


In [ ]:
plot_not_na = anime['plot_edited'][anime['plot_edited'].notna()]
plot_not_na

,plot_edited
0,"In the year 2071, roughly fifty years after an..."
1,"A few days before Halloween on Mars, Faye witn..."
2,"In the 32nd century, a man known as ""Vash the ..."
3,"Robin Sena is a ""craft user"", born in Japan an..."
4,"Set in the Dark Ages, a term referring to the ..."
...,...
14676,Extremely anxious and socially awkward Hitori ...
14688,"During Winter of his third middle school year,..."
14691,"Set in post-apocalyptic Japan, the land is rav..."
14704,High school student Miko Yotsuya has the unfor...


In [ ]:
short_plot = plot_not_na[plot_not_na.str.split(' ').apply(lambda x: len(x) <= 30)].index.tolist()
len(short_plot)

42

In [ ]:
anime.loc[short_plot, 'plot_edited'] = np.nan

In [ ]:
anime['plot'][anime['plot'].notna()].str.split(' ').apply(lambda x: len(x) > 500).sum()

np.int64(453)

In [ ]:
def comparison_func(df):

    if pd.isna(df['plot_edited']):
        return df['synopsis_edited']

    if pd.isna(df['synopsis_edited']):
        return df['plot_edited']

    len1 = len(df['plot_edited'].split(' '))
    len2 = len(df['synopsis_edited'].split(' '))
    if len1 >= len2:
        return df['plot_edited']
    else:
        return df['synopsis_edited']


In [ ]:
comp_results = anime.apply(comparison_func, axis = 1)
comp_results

,0
0,"In the year 2071, roughly fifty years after an..."
1,"A few days before Halloween on Mars, Faye witn..."
2,"In the 32nd century, a man known as ""Vash the ..."
3,"Robin Sena is a ""craft user"", born in Japan an..."
4,"Set in the Dark Ages, a term referring to the ..."
...,...
14703,NaN
14704,ko is a typical high school student whose life...
14705,NaN
14706,NaN


In [ ]:
anime['final_plot'] = comp_results

In [ ]:
anime['final_plot'].notna().sum()

np.int64(9262)

In [ ]:
content_df = anime.loc[anime['final_plot'].notna(),['MAL_ID', 'final_plot', 'Name']]

In [ ]:
content_df['final_plot'].shape

(9262,)

In [ ]:
content_df.reset_index(drop = True, inplace = True)

In [ ]:
content_df.head()

,MAL_ID,final_plot,Name
0,1,"In the year 2071, roughly fifty years after an...",Cowboy Bebop
1,5,"A few days before Halloween on Mars, Faye witn...",Cowboy Bebop: Tengoku no Tobira
2,6,"In the 32nd century, a man known as ""Vash the ...",Trigun
3,7,"Robin Sena is a ""craft user"", born in Japan an...",Witch Hunter Robin
4,8,"Set in the Dark Ages, a term referring to the ...",Bouken Ou Beet


In [ ]:
model = SentenceTransformer('Alibaba-NLP/gte-large-en-v1.5', trust_remote_code=True)

In [ ]:
content_df['plot_embeddings'] = content_df['final_plot'].apply(lambda x: model.encode(x, convert_to_tensor=False))

In [ ]:
import pickle
save_path = 'datasets/created_datasets/content_df_model.pkl'
with open(save_path, 'wb') as f:
    pickle.dump(content_df, f)